[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/geacomputing/UCY2Sept/blob/main/Python_code/Hands-on/6_Downloading_Data_from_Copernicus_API.ipynb)


In [ ]:
#Install packages
!rm -rf /content/sample_data


# Install netcdf library
!apt-get install -y netcdf-bin


# Clone the GitHub repo (replace with your fork if needed)
!git clone https://github.com/geacomputing/UCY2Sept.git
! pip install -r ./../../requirements.txt

In [ ]:
import cdsapi
import xarray
from datetime import datetime

## 🛠️ Exercise: Setting Up Your `.cdsapirc` File

Before using the **Copernicus CDS API**, you need to authenticate using your personal API key.  
This is done via a hidden config file called `.cdsapirc`, stored in your home directory.

---

### ✅ Step-by-Step Instructions

1. **Register** on the [Copernicus Climate Data Store (CDS)](https://cds.climate.copernicus.eu).
   - Create an account (if you don’t already have one).
   - After login, go to:  
     👉 [https://cds.climate.copernicus.eu/how-to-api)

2. **Copy your credentials**, which look like this:

    ```
    url: https://cds.climate.copernicus.eu/api/v2
    key: 123456:abcdef12-3456-7890-abcd-1234567890ab
    ```

3. **Use the Python code below** to create the `.cdsapirc` file with your credentials:

    ```python
    from pathlib import Path

    # Replace these with YOUR OWN credentials from the CDS website
    cds_url = "https://cds.climate.copernicus.eu/api"         # This is real
    cds_key = "123456:abcdef12-3456-7890-abcd-1234567890ab"   # This is made-up

    # (these are made up!) 

    # Build the config content
    cdsapirc_content = f"""url: {cds_url}
    key: {cds_key}
    """

    # Write to ~/.cdsapirc    
    config_path = Path.home() / ".cdsapirc" # <-------this is where the cdsapi expects to find the file!
    config_path.write_text(cdsapirc_content)

    print(f"✅ .cdsapirc file created at: {config_path}")
    ```



In [ ]:
from pathlib import Path

cds_url = "https://cds.climate.copernicus.eu/api"  
cds_key = ""

if not cds_key: 
    raise ValueError("Key is empty, before proceding you need to provide a vlid one!")
else:
    try: 
        cdsapirc_content = f"""url: {cds_url}
        key: {cds_key}
        """
        
        # Write to ~/.cdsapirc    
        config_path = Path.home() / ".cdsapirc" # <-------this is where the cdsapi expects to find the file!
        config_path.write_text(cdsapirc_content)
        
        print(f"✅ .cdsapirc file created at: {config_path}")
    except Exception as e:
        print(f"❌ Failed to write .cdsapirc file: {e}")
        

## Downloading ERA5 Temperature Data from Copernicus CDS API

In this section, we will demonstrate how to use the Copernicus Climate Data Store (CDS) Python API to programmatically request and download climate data.

Specifically, we will download **ERA5 reanalysis data** for 2-meter temperature over Cyprus for selected times on June 1, 2022.  

The example includes:  
- Specifying the dataset and parameters  
- Defining the geographic area (Cyprus bounding box)  
- Setting the date and time of interest  
- Requesting data in NetCDF format  
- Saving the downloaded file locally

Make sure you have your `.cdsapirc` file properly configured with your credentials before running this code.


In [ ]:
# Initialize the CDS API client
c = cdsapi.Client()

# Define parameters for the data request
variable = "2m_temperature"
year = "2022"
month = "06"
day = "01"
times = ["00:00", "06:00", "12:00", "18:00"]
area = [35.7, 32.2, 34.5, 34.0]  # [North, West, South, East] bounding box for Cyprus

# Create a dynamic filename
date_str = f"{year}{month}{day}"
filename = f"era5_{variable}_cyprus_{date_str}.nc" #add ".nc" for netcdf file

# Request data from CDS
c.retrieve(
    "reanalysis-era5-single-levels",
    {
        "product_type": "reanalysis",
        "format": "netcdf",
        "variable": variable,
        "year": year,
        "month": month,
        "day": day,
        "time": times,
        "area": area,
    },
    filename
)

print(f"Data successfully downloaded and saved as {filename}")